In [1]:
from __future__ import division
from __future__ import print_function
import torch
import time
import os
import glob
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from utils_new import *
from models import GAT
import _pickle as pickle
#from load_karate import load_karate
from sample_new import *
from sklearn.metrics import roc_auc_score
import numpy.core.numeric as _nx



In [2]:
# training settings
class Arguments():
  def __init__(self, cuda=True, fastmode=False, seed=42, epochs=100,
              lr=4e-4, weight_decay=5e-4, hidden=32, K=6, dropout=0.1, 
               batchSize=32, testBatchSize=32,
               sampleSize='20,20', breakPortion=0.2, patience="Patience", trainAttention=True):

    self.cuda = cuda
    self.K = K
    self.fastmode = fastmode
    self.seed = seed
    self.epochs = epochs
    self.lr = lr
    self.weight_decay = weight_decay
    self.hidden = hidden
    self.dropout = dropout
    self.batchSize = batchSize
    self.testBatchSize = testBatchSize
    self.sampleSize = sampleSize
    self.breakPortion = breakPortion
    self.patience = patience
    self.trainAttention = trainAttention

args = Arguments()
print(args)

In [3]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
ori_adj, adj, features, idx_train, idx_val, idx_test = load_project_data(args.breakPortion)

# set val set equivalent to test set

train negative sampling done


In [4]:
node_num = features.numpy().shape[0]
sampleSize = list(map(int, args.sampleSize.split(',')))

# model configuration 
model = GAT(K = 6, node_num = features.numpy().shape[0],nfeat=features.numpy().shape[1],nhid=args.hidden,nclass=2,sampleSize = sampleSize, dropout=args.dropout,trainAttention=args.trainAttention)

In [5]:
# set optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

if torch.cuda.device_count() > 1:
    print("Let's use multi-GPUs!")
    model = nn.DataParallel(model)
else:
    print("Only use one GPU")

labels = adj.view(node_num*node_num).type_as(idx_train)
labels = labels.type(torch.FloatTensor)
ori_labels = ori_adj.view(node_num*node_num).type_as(idx_train)
ori_labels = ori_labels.type(torch.FloatTensor)
criterion = nn.BCELoss() 
adj,ori_labels = Variable(adj), Variable(ori_labels) 
features = Variable(features)
labels = Variable(labels)

Only use one GPU


In [6]:
# copy data and model into GPU
if args.cuda:
    model = model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    criterion = criterion.cuda()
    ori_labels = ori_labels.cuda()


In [7]:
# create train, validation and test batches
train_batches = iterate_return(idx_train, args.sampleSize, labels, adj, 256)
val_batches   = iterate_return(idx_val, args.sampleSize, labels, adj, 256)
test_batches  = sample_test_batch(idx_test, args.sampleSize,adj, args.testBatchSize)

In [8]:
#train_batches = iterate_return_fun(idx_train, args.sampleSize, labels, adj, 32 )

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    batch = 0; loss_train_sum= 0; accu_train_sum = 0;loss_val_sum= 0; accu_val_sum = 0
    _idx_val, _idx_neighbor_val, _idx_neighbor2_val, _targets_val = val_batches[0]
    for epoch_cnt,i in enumerate(train_batches):
        _idx_train, _idx_neighbor_train, _idx_neighbor2_train, _targets_train = i
        output = model(torch.index_select(features,0,_idx_train),torch.index_select(features,0,_idx_neighbor_train),torch.index_select(features,0,_idx_neighbor2_train))      
        loss_train = criterion(output, _targets_train.type(torch.cuda.FloatTensor))
        acc_train = accuracy(output, _targets_train.type(torch.cuda.FloatTensor))
        model.train()
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        loss_train_sum += loss_train.item()
        accu_train_sum += acc_train.item()
        batch += 1

    if not args.fastmode:
        model.eval()
        output = model(torch.index_select(features,0,_idx_val),torch.index_select(features,0,_idx_neighbor_val),torch.index_select(features,0,_idx_neighbor2_val))
        loss_val = criterion(output, _targets_val.type(torch.cuda.FloatTensor))
        acc_val = accuracy(output, _targets_val.type(torch.cuda.FloatTensor))
        loss_val_sum += loss_val.item()
        accu_val_sum += acc_val.item()

    if (epoch +1)%10 ==0:
        test(epoch,test_batches)

    print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train_sum/float(batch)),
              'acc_train: {:.4f}'.format(accu_train_sum/float(batch)),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

    if args.cuda == True:
        return epoch+1, loss_train_sum/float(batch), loss_val.item(), accu_train_sum/float(batch), acc_val.item()
    else:
        return epoch+1, loss_train.data.numpy(), loss_val.data.numpy(), acc_train.data.numpy(), acc_val.data.numpy()

In [9]:
def test(epoch,test_batches):
    model.eval()
    batch = 0; loss_test_sum= 0; accu_test_sum = 0;f1_score_sum= 0; auc_sum = 0
    for i in test_batches:
        _idx_test, _idx_neighbor_test,_idx_neighbor2_test,targets  = i
        output = model(torch.index_select(features,0,_idx_test),torch.index_select(features,0,_idx_neighbor_test),torch.index_select(features,0,_idx_neighbor2_test))        
        loss_test = criterion(output, torch.index_select(ori_labels,0,targets))
        acc_test = accuracy(output,torch.index_select(ori_labels,0,targets).type(torch.cuda.FloatTensor))
        f1_score = score_f1(output,torch.index_select(ori_labels,0,targets).type(torch.cuda.FloatTensor))
        try:
            auc = roc_auc_score(torch.index_select(ori_labels,0,targets).cpu().data.numpy(),output.cpu().data.numpy())
        except:
            auc = 1

        loss_test_sum += loss_test.item()
        accu_test_sum += acc_test.item()
        f1_score_sum += f1_score
        auc_sum += auc
        batch += 1

    print("Test set results:",
          "loss= {:.4f}".format(loss_test_sum/float(batch)),
          "accuracy= {:.4f}".format(accu_test_sum/float(batch)),
          "f1_score = {:.4f}".format(f1_score_sum/float(batch)),
          "auc = {:.4f}".format(auc_sum/float(batch)))


In [10]:
def link_prediction():
    model.eval()
    output = model(features, adj)
    accu = link_prediction_accuracy_2label(ori_adj, adj, output)
    print ('link prediction accuracy is {}'.format(accu))


In [11]:
# Train model
t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):  # args.epochs
    epoch, loss_train, loss_val, acc_train, acc_val = train(epoch)
    loss_values.append(loss_val)
    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    torch.save(model, '{}.modelPkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break

    files = glob.glob('/content/Patience/*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)



Epoch: 0001 loss_train: 0.6958 acc_train: 0.4977 loss_val: 0.6934 acc_val: 0.4922 time: 2.4948s
Epoch: 0002 loss_train: 0.6649 acc_train: 0.5370 loss_val: 0.6280 acc_val: 0.6602 time: 1.3244s
Epoch: 0003 loss_train: 0.5429 acc_train: 0.7346 loss_val: 0.5217 acc_val: 0.7188 time: 1.3176s
Epoch: 0004 loss_train: 0.4792 acc_train: 0.7593 loss_val: 0.4892 acc_val: 0.7578 time: 1.3290s
Epoch: 0005 loss_train: 0.4536 acc_train: 0.7658 loss_val: 0.4754 acc_val: 0.7617 time: 1.3235s
Epoch: 0006 loss_train: 0.4451 acc_train: 0.7706 loss_val: 0.4675 acc_val: 0.7578 time: 1.3261s
Epoch: 0007 loss_train: 0.4369 acc_train: 0.7769 loss_val: 0.4620 acc_val: 0.7539 time: 1.3234s
Epoch: 0008 loss_train: 0.4320 acc_train: 0.7771 loss_val: 0.4594 acc_val: 0.7578 time: 1.3221s
Epoch: 0009 loss_train: 0.4274 acc_train: 0.7812 loss_val: 0.4564 acc_val: 0.7656 time: 1.3632s
Test set results: loss= 0.4232 accuracy= 0.7827 f1_score = 0.7827 auc = 0.8794
Epoch: 0010 loss_train: 0.4245 acc_train: 0.7818 loss_val

In [13]:
files = glob.glob('content/Patience/*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))
model_dict = model.state_dict()
test(epoch,test_batches)


Optimization Finished!
Total time elapsed: 431.8297s
Loading 84th epoch
Test set results: loss= 0.3651 accuracy= 0.8452 f1_score = 0.8452 auc = 0.9175
